In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.8 MB/s 
     |████████████████████████████████| 7.6 MB 21.9 MB/s 
     |████████████████████████████████| 182 kB 54.8 MB/s 


In [2]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [3]:
# example from https://towardsdatascience.com/clustering-the-20-newsgroups-dataset-with-gpt3-embeddings-10411a9ad150
import pandas as pd
from transformers import GPT2TokenizerFast
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train')
print(list(newsgroups_train.target_names))
print(newsgroups_train.data[:3])
print(newsgroups_train.target[:3])

df_news = pd.DataFrame(list(newsgroups_train.data), columns=['Text'])
df_news["Target"] = list(newsgroups_train.target)

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

df_news['n_tokens'] = df_news.Text.apply(lambda x: len(tokenizer.encode(x)))
df_news = df_news[df_news.n_tokens<2000]
print(len(df_news))

df_news.head(5)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever in

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1119 > 1024). Running this sequence through the model will result in indexing errors


10995


,Text,Target,n_tokens
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,211
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,238
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,506
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,271
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,326


In [4]:
import numpy as np

In [5]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2')  # or any other checkpoint
word_embeddings = model.transformer.wte.weight  # Word Token Embeddings 
position_embeddings = model.transformer.wpe.weight  # Word Position Embeddings

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [6]:
import pandas as pd
df = pd.read_csv("/drive/My Drive/Colab Notebooks/data/IST 736 projects/teaching tools/teaching.tools.feedback.csv")

In [7]:
docs = df[~df['teaching.tools.feedback'].isna()]
corpus = df[~df['teaching.tools.feedback'].isna()]

In [8]:
docs = docs['teaching.tools.feedback']
corpus = corpus['teaching.tools.feedback']

In [9]:
#gpt_embeddings = word_embeddings.encode(corpus)

In [10]:
docs.head()

0    See feedback from earlier comment about Blackb...
1                        Timing of changes is terrible
2    Access to Blackboard support is challenging. O...
3    International students accessing class and mat...
4    I have had some unique challenges with playpos...
Name: teaching.tools.feedback, dtype: object

In [11]:
crTokens = docs.apply(lambda x: len(tokenizer.encode(x)))
# df_news['n_tokens'] = df_news.Text.apply(lambda x: len(tokenizer.encode(x)))

In [12]:
crTokens

0       9
1       6
2      21
3      20
4      31
       ..
142    14
143     2
144    25
145    54
146    96
Name: teaching.tools.feedback, Length: 147, dtype: int64

In [13]:
cr = pd.DataFrame(docs, crTokens)

In [14]:
cr.head()

,teaching.tools.feedback
teaching.tools.feedback,
9,I feel that there are new teaching technology ...
6,I feel like the online teaching group has been...
21,I have answered in terms of the central ITS st...
20,One of the things that I love about SU is the ...
31,Sending faculty to Answers for every issue is ...


In [15]:
cr = pd.DataFrame(docs)

In [16]:
cr.head()

,teaching.tools.feedback
0,See feedback from earlier comment about Blackb...
1,Timing of changes is terrible
2,Access to Blackboard support is challenging. O...
3,International students accessing class and mat...
4,I have had some unique challenges with playpos...


In [17]:
cr = cr.merge(crTokens, left_index = True, right_index = True)

In [18]:
cr.head()

,teaching.tools.feedback_x,teaching.tools.feedback_y
0,See feedback from earlier comment about Blackb...,9
1,Timing of changes is terrible,6
2,Access to Blackboard support is challenging. O...,21
3,International students accessing class and mat...,20
4,I have had some unique challenges with playpos...,31


In [19]:
cr = cr.reset_index()

In [20]:
cr.head()

,index,teaching.tools.feedback_x,teaching.tools.feedback_y
0,0,See feedback from earlier comment about Blackb...,9
1,1,Timing of changes is terrible,6
2,2,Access to Blackboard support is challenging. O...,21
3,3,International students accessing class and mat...,20
4,4,I have had some unique challenges with playpos...,31


In [21]:
cr = cr.drop(columns = ['index'])
cr = cr.rename({"teaching.tools.feedback_x" : "teaching_tools_feedback_text"}, axis = 1)
cr = cr.rename({"teaching.tools.feedback_y" : "teaching_tools_feedback_token"}, axis = 1)

In [22]:
cr.head()

,teaching_tools_feedback_text,teaching_tools_feedback_token
0,See feedback from earlier comment about Blackb...,9
1,Timing of changes is terrible,6
2,Access to Blackboard support is challenging. O...,21
3,International students accessing class and mat...,20
4,I have had some unique challenges with playpos...,31


In [23]:
crTokens = crTokens.reset_index()

In [24]:
crTokens = crTokens.drop(columns = ['index'])

In [25]:
crVector = model.transformer.wte.weight[crTokens['teaching.tools.feedback']]

In [26]:
crTokens

,teaching.tools.feedback
0,9
1,6
2,21
3,20
4,31
...,...
142,14
143,2
144,25
145,54


In [27]:
crVector

tensor([[-0.0401,  0.0523,  0.1221,  ...,  0.0295, -0.0433,  0.0227],
        [-0.0839,  0.0319,  0.0684,  ..., -0.0647, -0.1318,  0.1108],
        [-0.0410, -0.0706,  0.1606,  ...,  0.0348,  0.1737, -0.0178],
        ...,
        [ 0.0410, -0.1835,  0.0336,  ...,  0.0241, -0.0834, -0.0254],
        [ 0.0337, -0.1199,  0.2371,  ..., -0.0661, -0.0174, -0.1863],
        [ 0.1482, -0.0096,  0.2772,  ...,  0.0086,  0.0135,  0.1423]],
       grad_fn=<IndexBackward0>)

In [28]:
crVectorDetach = crVector.detach()

In [29]:
crVectorDetach

tensor([[-0.0401,  0.0523,  0.1221,  ...,  0.0295, -0.0433,  0.0227],
        [-0.0839,  0.0319,  0.0684,  ..., -0.0647, -0.1318,  0.1108],
        [-0.0410, -0.0706,  0.1606,  ...,  0.0348,  0.1737, -0.0178],
        ...,
        [ 0.0410, -0.1835,  0.0336,  ...,  0.0241, -0.0834, -0.0254],
        [ 0.0337, -0.1199,  0.2371,  ..., -0.0661, -0.0174, -0.1863],
        [ 0.1482, -0.0096,  0.2772,  ...,  0.0086,  0.0135,  0.1423]])

In [31]:
from sklearn.cluster import KMeans
K =9  # change number of cluster if applicable
gpt_model = KMeans(n_clusters = K, random_state = 2022)
gpt_model.fit(crVectorDetach)
labels = gpt_model.labels_.tolist()  # Save cluster labels in a separate list

In [32]:
def print_docs_closest_to_centroids(model, vec, n):
  K = len(model.cluster_centers_)
  for j in range(K):
    d = model.transform(vec)[:, j] # transform all docs to cluster-distance space
    idx = np.argsort(d)[:n] # find n docs closest to centroid

    c_idx = [m for m, label in enumerate(model.labels_) if label==j]  # find the index of all docs in cluster j   
    print('\n\n======cluster #', j, ', cluster size:', len(c_idx))
    for i in idx:
      if i not in c_idx:
        print('[ this doc is in a different cluster #', model.labels_[i], '>>', corpus[i])
      else:
        print(corpus[i])
  return

In [33]:
def print_cluster_sizes(model):
  cnt_perCluster = {}
  for c in model.labels_:
    cnt_perCluster[c] = cnt_perCluster.get(c, 0) + 1
  print(cnt_perCluster)

In [34]:
print_cluster_sizes(gpt_model)

{8: 8, 2: 34, 1: 26, 6: 5, 3: 35, 7: 7, 5: 16, 4: 11, 0: 5}


In [35]:
print_docs_closest_to_centroids(gpt_model, crVectorDetach, 5)



======cluster # 0 , cluster size: 5
Even after using Blackboard for over a decade, it still feels cumbersome and unreliable. The ITS help desk are good at trouble shooting but when I call it often feels as if they are understaffed.
This may be my issue, I don't always see the updates from IT on changes to the systems. I was surprised when I went to upload a new course and there were changes to Blackboard. 
Sign on to the "system" with my credentials every single time I want to use Adobe in a classroom computer to show my slides...really?  This is too burdensome and a drag on productivity!
In general, very supportive of the ITS team. Just feel as though they need more resources to do their job well, inclusive of getting faculty to think about the best case use stories for these important technologies. 
Blackboard may not be the best tool out there, but it's getting the job done and changing it for a marginal gain doesn't make much sense. SU ITS does a great job in helping faculty depl

In [ ]:
crVectorDetach.size()

torch.Size([69, 768])